In [55]:
from tools import *
import boto3
brt = boto3.client('bedrock-runtime')

## ユーザーのクエリを深ぼる

In [56]:
USER_PROMPT = 'きのこの山とたけのこの里、どちらが至高のお菓子かの結論'
MODEL_ID_DICT = {
    'claude-3.5-sonnet': 'us.anthropic.claude-3-5-sonnet-20241022-v2:0',
    'nova-pro': 'us.amazon.nova-pro-v1:0',
    'mistral-large': 'mistral.mistral-large-2407-v1:0',
    'llama-3.3' : 'us.meta.llama3-3-70b-instruct-v1:0'
}

# 使用するモデルのキーを設定
MODEL_A = 'claude-3.5-sonnet'  # 1つ目のAI
MODEL_I = 'llama-3.3'      # 2つ目のAI

MAX_LOOP = 5

QUERY_QUALIFICATION_SYSTEM_PROMPTS = [{
    'text':f'''あなたは優秀なリサーチャーです。
会話相手はあなたと同じ {USER_PROMPT} という調査依頼を受けとった同僚の AI さんです。
調査内容は雑なので、調査の粒度や観点などは仮説を持って調査をした後調査結果を作成し、調査結果のフィードバックをもらうことでしか改善できません。
AI さんはあなたの思考の枠を外して広い視野を提供してくれます。
あなたも調査内容を自由に広げて網羅性を高め、その後どんなことを調べるのかを深めていってください。
特に反対意見は大事です。お互いの網羅性や深さの不足を指摘しながらAI さんとの会話を重ね、リサーチする内容を決めていってください。
会話はお互い {MAX_LOOP} 回までしかできないので、それまでに議論をまとめてください。
ただし会話は <rules> で与える内容をまとめてください。
<rules>
* 具体的にどんなことをするのか actionable にまとめる必要があります。
* 予算や人員については触れてはいけません。調査する内容にだけフォーカスしてください。
* 調査対象に関わるだろう人の観点を複数入れてください。例えば料理であれば、調理器具をつくる人、食材を運ぶ人、料理を作る人、料理を運ぶ人、食べる人、口コミを書く人などです。与えられたお題に反しない限り様々な人に思いを巡らせてください。
* 会話を始める前に、自分がどのように考えたのか、を述べてから結論を述べてください。
* さまざまな観点から内容をブラッシュアップしてください。
</rules>
また、発言する際は最初に必ず x 回目の発言です、と言ってください。発言回数は自分の発言回数であり、相手の発言はカウントしてはいけません。
'''}]

conversation = {
    'A':[
        {
            "role": "user",
            "content": [{"text": f'「{USER_PROMPT}」って大変なお題をもらっちゃいましたね。どうしましょうか。'}]
        }
    ],
    'I':[]
}

# 5往復のループ
for i in range(MAX_LOOP):
    # 1つ目のAIの応答
    response = brt.converse(
        modelId=MODEL_ID_DICT[MODEL_A],
        messages=conversation['A'],
        system=QUERY_QUALIFICATION_SYSTEM_PROMPTS,
        inferenceConfig={'temperature': 1},
    )
    first_response = response['output']['message']['content'][0]['text']
    print(f"\n=== {MODEL_A}:{i+1} 回目の発言 ===\n")
    print(first_response)
    
    
    conversation['A'].append(
        {
            "role": "assistant",
            "content": [{"text": first_response}]
        }
    )
    conversation['I'].append(
        {
            "role": "user",
            "content": [{"text": first_response}]
        }
    )

    # 2つ目のAIの応答
    response = brt.converse(
        modelId=MODEL_ID_DICT[MODEL_I],
        messages=conversation['I'],
        system=QUERY_QUALIFICATION_SYSTEM_PROMPTS,
        inferenceConfig={'temperature': 1},
    )
    second_response = response['output']['message']['content'][0]['text']
    print(f"\n=== {MODEL_I}:{i+1} 回目の発言 ===\n")
    print(second_response)
    
    # 次の往復のために会話履歴を更新
    conversation['I'].append(
        {
            "role": "assistant",
            "content": [{"text": second_response}]
        }
    )
    conversation['A'].append(
        {
            "role": "user",
            "content": [{"text": second_response}]
        }
    )


=== claude-3.5-sonnet:1 回目の発言 ===

1回目の発言です。

まず、この調査の難しさを整理してみました：
1. 好みの主観性が強い題材である
2. 長年の論争があり、感情的な意見も多い
3. 「至高」の定義自体が曖昧

そこで、以下のような調査アプローチを提案します：

【製品サイドの調査】
- 製造工程の複雑さと技術的革新性
- 原材料の品質と調達の持続可能性
- パッケージデザインの心理的影響
- 製造者の想いと開発秘話

【消費者サイドの調査】
- 年齢層別の嗜好性調査
- 食べ方の多様性（一口で食べる/分解して食べるなど）
- SNSでの言及傾向分析
- 購買層の特徴分析

【流通サイドの調査】
- 販売店での陳列方法の効果
- 季節による売れ行きの変化
- コンビニ/スーパー/菓子専門店での販売傾向

【文化的側面の調査】
- メディアでの取り上げられ方
- 菓子文化における位置づけ
- 地域による受容度の違い

いかがでしょうか？ただ、これだと表面的すぎる気もしています。もっと深い視点や、見落としている観点があるかもしれません。

=== llama-3.3:1 回目の発言 ===

2回目の発言です。

あなたの提案は、きのこの山とたけのこの里の至高性を調査するための幅広い観点を網羅しています。特に、製品サイド、消費者サイド、流通サイド、文化的側面の調査は、多角的な視点からこの問題にアプローチするための良い出発点です。

ただし、以下の点を追加することで、調査の深さと網羅性を高めることができます：

* **感覚評価**: 消費者がきのこの山とたけのこの里を食べたときの感覚的な体験（味、香り、テクスチャなど)を評価する調査を追加することで、至高性の主観的な側面をより具体的に捉えることができます。
* **比較分析**: きのこの山とたけのこの里以外の類似菓子との比較分析を実施することで、どのような特徴が至高性に寄与しているのかを明らかにすることができます。
* **歴史的背景**: きのこの山とたけのこの里の歴史的背景、つまりどのようにして開発され、どのようにして人気を博したのかを調査することで、至高性の根底にある要因を理解することができます。
* **心理学的側面**: 消費者がきのこの山とたけのこの里を食べる際の心理

## 調査すべき内容、アプローチをまとめる

In [57]:
SURVEY_STRATEGY_PROMPTS = [{
    'text':f'''あなたは優秀なリサーチャーです。
あなたは「{USER_PROMPT}」 という調査依頼を受けとっています。
調査内容は雑なので、調査の粒度や観点などは仮説を持って調査をした後調査結果を作成し、調査結果のフィードバックをもらうことでしか改善できません。
どのように調査を進めるかの方針をまとめた会話を渡します。
調査の方針をまとめてください。
ただし出力する内容は調査の方針だけで、会話を続ける必要はありません。
'''}]

In [58]:
response = brt.converse(
    modelId=MODEL_ID_DICT[MODEL_A],
    messages=conversation['A'],
    system=SURVEY_STRATEGY_PROMPTS,
    inferenceConfig={'temperature': 0},
)
report_framework_response = response['output']['message']['content'][0]['text']
print(report_framework_response)

調査方針をまとめます：

【調査の基本方針】
「きのこの山とたけのこの里の至高性」を、以下の5つの価値軸から多面的に評価する

1. 商品価値の評価
- 食品工学的特性（味覚、食感、保存性）
- 製造工程・原材料の分析
- パッケージング機能性

2. 市場価値の評価
- 国内外の売上動向
- 価格競争力分析
- 流通経路別実績

3. 消費者価値の評価
- 官能評価（味、香り、食感）
- 喫食シーン分析
- 感情価値の定量化
- SNS・メディアでの評価

4. 社会的価値の評価
- 環境負荷
- 持続可能性
- 文化的貢献度
- 食育としての価値

5. グローバル価値の評価
- 海外市場での受容性
- 文化的適応性
- 長期的発展可能性
- ブランド価値の持続性

【調査の特徴】
- 客観性：定量的評価基準の採用
- 包括性：多面的な評価軸の設定
- 実行可能性：具体的な調査項目の明確化
- 汎用性：他製品評価への応用可能性
- 発展性：時代や地域変化への対応


In [59]:
SURVEY_STRATEGY_PROMPTS = [{
    'text':f'''あなたは優秀なリサーチャーです。
あなたは「{USER_PROMPT}」 という調査依頼を受けとっています。
ユーザーはレポートのフレームワーク与えます。
ただしあなたは Web 検索をするか、検索結果の URL にアクセスをして情報を取得し、その結果を元に自分自身で考察する以外のことはできません。
あなたは、どうやって調査を進めるか考えたかと保持しているツールを使用して、必要な情報をすべて集めてください。
'''}]

In [60]:
tools = []
tools.append(
    {
        'toolSpec': {
            'name': 'search',
            'description': '''検索する文章、キーワードを受け取って Web 検索する''',
            'inputSchema': {
                'json': {
                    'type': 'object',
                    'properties': {
                        'query': {
                            'type': 'string',
                            'description': '検索する文章またはキーワード。半角スペースで区切ることで複数のキーワードを受け付ける。',
                        }
                    },
                    'required': ['query'],
                }
            },
        }
    }
)
tools.append(
    {
        'toolSpec': {
            'name': 'get_content',
            'description': '''URL にアクセスして html のコンテンツを取得''',
            'inputSchema': {
                'json': {
                    'type': 'object',
                    'properties': {
                        'url': {
                            'type': 'string',
                            'description': '情報を取得したい URL',
                        }
                    },
                    'required': ['url'],
                }
            },
        }
    }
)
tools.append(
    {
        'toolSpec': {
            'name': 'is_finished',
            'description': '''ツールの利用が完了し、次のステップに進む関数''',
            'inputSchema': {
                'json': {
                    'type': 'object',
                    'properties': {},
                    'required': [],
                }
            },
        }
    }
)
tool_config = {'tools': tools}

In [61]:
conversation['F'] = []
conversation['F'].append(
    {
        "role": "user",
        "content": [{"text": report_framework_response}]
    }
)
print(conversation['F'])

[{'role': 'user', 'content': [{'text': '調査方針をまとめます：\n\n【調査の基本方針】\n「きのこの山とたけのこの里の至高性」を、以下の5つの価値軸から多面的に評価する\n\n1. 商品価値の評価\n- 食品工学的特性（味覚、食感、保存性）\n- 製造工程・原材料の分析\n- パッケージング機能性\n\n2. 市場価値の評価\n- 国内外の売上動向\n- 価格競争力分析\n- 流通経路別実績\n\n3. 消費者価値の評価\n- 官能評価（味、香り、食感）\n- 喫食シーン分析\n- 感情価値の定量化\n- SNS・メディアでの評価\n\n4. 社会的価値の評価\n- 環境負荷\n- 持続可能性\n- 文化的貢献度\n- 食育としての価値\n\n5. グローバル価値の評価\n- 海外市場での受容性\n- 文化的適応性\n- 長期的発展可能性\n- ブランド価値の持続性\n\n【調査の特徴】\n- 客観性：定量的評価基準の採用\n- 包括性：多面的な評価軸の設定\n- 実行可能性：具体的な調査項目の明確化\n- 汎用性：他製品評価への応用可能性\n- 発展性：時代や地域変化への対応'}]}]


In [62]:
for i in range(100):
    print(i)
    response = brt.converse(
        modelId=MODEL_ID_DICT[MODEL_A],
        messages=conversation['F'],
        system=SURVEY_STRATEGY_PROMPTS,
        inferenceConfig={'temperature': 0},
        toolConfig=tool_config
    )
    conversation['F'].append({
        'role': 'assistant', 
        'content': response['output']['message']['content']
    })
    
    # tool_useを初期化
    tool_use = None
    
    # try-exceptブロックで例外処理を追加
    try:
        content_list = response.get('output', {}).get('message', {}).get('content', [])
        # contentリストの各要素を確認
        for content_item in content_list:
            if isinstance(content_item, dict) and 'toolUse' in content_item:
                tool_use = content_item['toolUse']
                break
    except (AttributeError, TypeError):
        break
        
    if tool_use is not None:
        function_name = tool_use['name']
        if function_name == 'is_finished':
            conversation['F'].append({
                'role': 'user', 
                'content': [
                    {
                        'toolResult':{
                            'toolUseId':tool_use['toolUseId'],
                            'content':[
                                {
                                    'text':'finished'
                                }
                            ]
                        }
                    }
                ]
            })
            break
        function_args = tool_use['input']
        
        result = globals()[function_name](**function_args)
        conversation['F'].append({
            'role': 'user', 
            'content': [
                {
                    'toolResult':{
                        'toolUseId':tool_use['toolUseId'],
                        'content':[
                            {
                                'text':result
                            }
                        ]
                    }
                }
            ]
        })
    else:
        break

0
1
2
3
4
5


In [63]:
extracted_text = ""

for c in conversation['F']:
    if 'content' in c:
        for item in c['content']:
            if 'text' in item:
                extracted_text += item['text'] + "\n\n"
            elif 'toolResult' in item and 'content' in item['toolResult']:
                for content_item in item['toolResult']['content']:
                    if 'text' in content_item:
                        extracted_text += content_item['text'] + "\n\n"

# 結果の確認（必要な場合）
print(extracted_text)

調査方針をまとめます：

【調査の基本方針】
「きのこの山とたけのこの里の至高性」を、以下の5つの価値軸から多面的に評価する

1. 商品価値の評価
- 食品工学的特性（味覚、食感、保存性）
- 製造工程・原材料の分析
- パッケージング機能性

2. 市場価値の評価
- 国内外の売上動向
- 価格競争力分析
- 流通経路別実績

3. 消費者価値の評価
- 官能評価（味、香り、食感）
- 喫食シーン分析
- 感情価値の定量化
- SNS・メディアでの評価

4. 社会的価値の評価
- 環境負荷
- 持続可能性
- 文化的貢献度
- 食育としての価値

5. グローバル価値の評価
- 海外市場での受容性
- 文化的適応性
- 長期的発展可能性
- ブランド価値の持続性

【調査の特徴】
- 客観性：定量的評価基準の採用
- 包括性：多面的な評価軸の設定
- 実行可能性：具体的な調査項目の明確化
- 汎用性：他製品評価への応用可能性
- 発展性：時代や地域変化への対応

承知しました。非常に体系的な調査フレームワークですね。与えられたツールを使って、各評価軸に関する情報を収集していきましょう。

まず、基本的な情報から収集を始めます：

1. 両製品の基本情報：

[{"title": "きのこの山/たけのこの里", "url": "https://www.meiji.com/100th/kinotake.html", "description": "「<strong>きのこの山</strong>」の誕生...ことから、のどかな里山をイメージしたものにした。そして、「<strong>たけのこの里</strong>」の発売は1979年。「<strong>きのこの山</strong>」に続く、第2弾として生まれた商品である。「<strong>きのこの山</strong>」がクラ..."}, {"title": "【きのこの山vsたけのこの里】どっちが人気？国民的お菓子は永遠のライバル | ツギノジダイ", "url": "https://smbiz.asahi.com/article/14617374", "description": "国民の祝日「山の日」が制定されたことに便乗し、明治が2016年に定めたのが、8月11日の「きのこの山の日」。 · 2017年には、3月1

In [64]:
messages = [{"role": "user","content": [{"text": f'{extracted_text}\n\nこれでレポートを書いて。'}]}]

GENERATE_REPORT_PROMPT = f'''あなたは優秀なリサーチャーです。
あなたは「{USER_PROMPT}」 という調査依頼を受けとっています。
そして、調査の仕方については「{report_framework_response}」で定義されています。
必要な情報はユーザーが提供します。
マークダウン形式のレポートを作成してください。ただしナレーティブに詳細を書いてください。
'''

In [65]:
response = brt.converse(
    modelId=MODEL_ID_DICT[MODEL_A],
    messages=messages,
    system=SURVEY_STRATEGY_PROMPTS,
    inferenceConfig={'temperature': 1},
)
final_report = response['output']['message']['content'][0]['text']
print(final_report)

収集した情報を基に、各評価軸について分析したレポートを作成いたします。

【きのこの山とたけのこの里 至高性分析レポート】

1. 商品価値の評価

■食品工学的特性：
- きのこの山：
  - カカオチョコレートを使用し、甘さ控えめで大人向けの味わい
  - クラッカーベースで歯ごたえのある食感
  - チョコとビスケットの二層構造による複雑な味わい

- たけのこの里：
  - ミルクチョコレートを使用し、まろやかで親しみやすい味わい
  - クッキー生地による柔らかな食感
  - 卵を使用した生地による優しい口溶け

■製造工程・原材料：
- きのこの山：アポロの製造ライン活用から開発（1975年）
- たけのこの里：型焼き製法による独自の形状（1979年）
- 両製品とも安定した品質管理と大量生産体制を確立

2. 市場価値の評価

■売上動向：
- 直近の消費動向では「たけのこの里」が優位
- 年齢層による購買傾向の差異
  - たけのこの里：45歳まで安定した人気（3-6位を維持）
  - きのこの山：若年層での人気が相対的に低い

■価格競争力：
- 同価格帯での競争
- 2023年以降、原材料高騰による内容量約1割減

3. 消費者価値の評価

■年齢層別評価：
- 若年層：たけのこの里が優位
  - マイルドな味わいと柔らかい食感が好評
- 成人層：きのこの山の支持が増加
  - カカオの風味と食感の複雑さを評価

■SNS・メディアでの評価：
- 「きのこ派」「たけのこ派」の二項対立による話題性
- 2001年の総選挙ではたけのこ派が勝利

4. 社会的価値の評価

■環境への取り組み：
- 「きのこの山・たけのこの里の森」での森林保全活動
- パッケージの環境配慮

■文化的貢献：
- 日本の菓子文化における革新的な商品として確立
- チョコスナックという新ジャンルの開拓

5. グローバル価値の評価

■海外展開：
- きのこの山は「CHOCOROOMS」として米国展開（2007年）
- 日本発のユニークな菓子としての認知

【結論】
両商品はそれぞれ異なる価値を持つ優れた商品であり、一方的な優劣を付けることは適切ではない。

たけのこの里は：
- より広い年齢層での支持
- 安定した市場シェア
- 親しみやすい味わい
が特徴

きのこの山は